In [ ]:
import requests
from datetime import datetime, timedelta
import time
import json
import pandas as pd
import pytz
from uuid import uuid4

# NASDAQ 100 data only

In [ ]:
# API Key
api_key = '#################################'

# nasdaq100_ticker_dataset.json tickers
with open("nasdaq100_ticker_dataset.json", "r") as file:
    data = json.load(file)
    tickers = [item["Ticker"] for item in data]

print(f"{len(tickers)} tickers loaded from nasdaq100_ticker_dataset.json")

date_from = '2020-01-01'
date_to = '2025-02-17'

# API request handler with pagination, rate limiting, and detailed logging
def fetch_data(url, feature_name, ticker=None, max_retries=3):
    """Handles API requests with pagination, rate limiting, and detailed logging"""
    all_results = []
    retry_count = 0
    
    while url and retry_count < max_retries:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                # Log response details
                print(f"API Response for {feature_name} ({ticker or 'GENERAL'}):")
                print(f"Status: {data.get('status')}")
                print(f"Query Count: {data.get('queryCount')}")
                print(f"Results Count: {data.get('resultsCount')}")
                print(f"Next URL: {data.get('next_url')}")
                
                if 'results' in data:
                    all_results.extend(data['results'])
                
                # Handling pagination
                url = data.get('next_url', None)
                if url:
                    print(f"Fetching next page for {feature_name} on {ticker or 'GENERAL'}...")
                    url = f"{url}&apiKey={api_key}"  # Appending API key to next_url
                    retry_count = 0  # Resetting retry count for new page
            elif response.status_code == 429:  # Too many requests
                print(f"API rate limit exceeded. Waiting 60 seconds...")
                time.sleep(60)
                retry_count += 1
            else:
                print(f"Error {response.status_code} for {feature_name} on {ticker or 'GENERAL'}: {response.text}")
                retry_count += 1
                time.sleep(5)  # Small delay before retrying
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            retry_count += 1
            time.sleep(5)  # Small delay before retrying
    
    if retry_count >= max_retries:
        print(f"Max retries reached for {feature_name} on {ticker or 'GENERAL'}")
    
    return {'results': all_results} if all_results else {}

# Step 1: Fetching OHLCV Data & Save
ohlcv_data = []
missing_tickers = []  
available_start_date = set()
available_end_date = set()

for ticker in tickers:
    print(f"Fetching OHLCV data for {ticker}...")
    url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date_from}/{date_to}?apiKey={api_key}'
    stock_data = fetch_data(url, "Stock Data", ticker)
    
    if not stock_data or 'results' not in stock_data or not stock_data['results']:
        print(f"No OHLCV data for {ticker}, skipping...")
        missing_tickers.append(ticker)
        continue  

    dates_available = [pd.to_datetime(item['t'], unit='ms').date() for item in stock_data['results']]
    min_date = min(dates_available)
    max_date = max(dates_available)
    print(f"Data for {ticker} spans {min_date} to {max_date}")
    if min_date > pd.to_datetime(date_from).date() or max_date < pd.to_datetime(date_to).date():
        print(f"Warning: Incomplete date range for {ticker}. Expected {date_from} to {date_to}")

    # Identifying available dates
    dates_available_set = set(dates_available)
    if pd.to_datetime(date_from).date() in dates_available_set:
        available_start_date.add(ticker)
    if pd.to_datetime(date_to).date() in dates_available_set:
        available_end_date.add(ticker)

    # Storing OHLCV data
    for minute_data in stock_data['results']:
        row = {
            "Date": pd.to_datetime(minute_data['t'], unit='ms').date(),
            "Time": pd.to_datetime(minute_data['t'], unit='ms').time(),
            "Ticker": ticker,
            "Open": minute_data['o'],
            "High": minute_data['h'],
            "Low": minute_data['l'],
            "Close": minute_data['c'],
            "Volume": minute_data['v']
        }
        ohlcv_data.append(row)

# Converting and saving
df = pd.DataFrame(ohlcv_data)
df.to_parquet("backtest_data_step1.parquet", index=False)

with open("missing_tickers.json", "w") as f:
    json.dump(missing_tickers, f, indent=4)

print(f"Total tickers processed: {len(tickers)}")
print(f"Total tickers missing data: {len(missing_tickers)}")
print(f"Tickers with data from start date: {len(available_start_date)}")
print(f"Tickers with data to end date: {len(available_end_date)}")

# Step 2: Computing Previous Session High/Low & Saving
df = pd.read_parquet("backtest_data_step1.parquet")
df.sort_values(by=["Ticker", "Date", "Time"], inplace=True)

daily_high_low = df.groupby(["Ticker", "Date"]).agg(
    Prev_Session_High=("High", "max"),
    Prev_Session_Low=("Low", "min")
).reset_index()

daily_high_low["Prev_Trading_Date"] = daily_high_low.groupby("Ticker")["Date"].shift(1)
daily_high_low["Prev_Trading_Date"] = daily_high_low.groupby("Ticker")["Prev_Trading_Date"].ffill()

df = df.merge(
    daily_high_low[["Ticker", "Date", "Prev_Trading_Date"]],
    on=["Ticker", "Date"],
    how="left"
)

df = df.merge(
    daily_high_low[["Ticker", "Date", "Prev_Session_High", "Prev_Session_Low"]],
    left_on=["Ticker", "Prev_Trading_Date"],
    right_on=["Ticker", "Date"],
    how="left",
    suffixes=("", "_Prev")
)

df.drop(columns=["Prev_Trading_Date", "Date_Prev"], inplace=True)
first_day = df["Date"].min()
df = df[df["Date"] != first_day]

df.to_parquet("backtest_data_step2.parquet", index=False)

# Step 3: Computing Bid-Ask Spread & OBD & Saving
def compute_bid_ask_spread(df):
    df["Bid-Ask Spread (Estimated)"] = df["High"] - df["Low"]
    df["Bid-Ask Spread (Estimated)"] = df["Bid-Ask Spread (Estimated)"].apply(lambda x: max(x, 0.0001))
    return df

def compute_obd(df):
    if "Bid-Ask Spread (Estimated)" not in df.columns:
        df = compute_bid_ask_spread(df)
    df["Estimated OBD"] = df["Volume"] / df["Bid-Ask Spread (Estimated)"]
    return df

df = pd.read_parquet("backtest_data_step2.parquet")
df = compute_bid_ask_spread(df)
df = compute_obd(df)
df.to_parquet("backtest_data_step3.parquet", index=False)

# Step 4: Computing 50-day SMA using daily close prices and saving
df = pd.read_parquet("backtest_data_step3.parquet")
df["Date"] = pd.to_datetime(df["Date"])

daily_closes = df.groupby(["Ticker", "Date"])["Close"].last().reset_index()
daily_closes["50-day SMA"] = daily_closes.groupby("Ticker")["Close"].transform(lambda x: x.rolling(window=50, min_periods=1).mean())

df = df.merge(daily_closes[["Ticker", "Date", "50-day SMA"]], on=["Ticker", "Date"], how="left")
df = df[df["Date"] >= "2020-02-17"]

df.to_parquet("backtest_data_step4.parquet", index=False)

# Backtest data - 5 years back
https://github.com/datasets/nasdaq-listings/tree/main   -> Tickers

In [ ]:
# API Key
api_key = '####################################'

# Loading tickers from json
with open("nasdaq_tickers_17_02_2025.json", "r") as file:
    tickers = json.load(file) # Limit to first ticker for testing [11:12]  for AAPL

date_from = '2020-01-01'
date_to = '2025-02-17'

# API request handler with rate limiting
def fetch_data(url, feature_name, ticker=None, max_retries=3):
    """Handles API requests with rate limiting and error logging"""
    all_results = []
    retry_count = 0
    
    while url and retry_count < max_retries:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                if 'results' in data:
                    all_results.extend(data['results'])
                
                # Handling pagination
                url = data.get('next_url', None)
                if url:
                    print(f"Fetching next page for {feature_name} on {ticker or 'GENERAL'}...")
                    url = f"{url}&apiKey={api_key}"  # Appending API key to next_url
                    retry_count = 0  # Resetting retry count for new page
            elif response.status_code == 429:  # Too many requests
                print(f"API rate limit exceeded. Waiting 60 seconds...")
                time.sleep(60)
                retry_count += 1
            else:
                print(f"Error {response.status_code} for {feature_name} on {ticker or 'GENERAL'}: {response.text}")
                retry_count += 1
                time.sleep(5)  # Small delay before retrying
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            retry_count += 1
            time.sleep(5)  # Small delay before retrying
    
    if retry_count >= max_retries:
        print(f"Max retries reached for {feature_name} on {ticker or 'GENERAL'}")
    
    return {'results': all_results} if all_results else {}


### Step 1: Fetching OHLCV data

In [ ]:
ohlcv_data = []
missing_tickers = []  # Tracking missing tickers
available_start_date = set()
available_end_date = set()

for ticker in tickers:
    print(f"Fetching OHLCV data for {ticker}...")
    url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date_from}/{date_to}?apiKey={api_key}'
    stock_data = fetch_data(url, "Stock Data", ticker)
    
    if not stock_data or 'results' not in stock_data:
        print(f"No OHLCV data for {ticker}, skipping...")
        missing_tickers.append(ticker)
        continue  

    # Identifying available dates
    dates_available = set(pd.to_datetime(item['t'], unit='ms').date() for item in stock_data['results'])
    
    if pd.to_datetime(date_from).date() in dates_available:
        available_start_date.add(ticker)
    if pd.to_datetime(date_to).date() in dates_available:
        available_end_date.add(ticker)

    # Storing OHLCV data
    for minute_data in stock_data['results']:
        row = {
            "Date": pd.to_datetime(minute_data['t'], unit='ms').date(),
            "Time": pd.to_datetime(minute_data['t'], unit='ms').time(),
            "Ticker": ticker,
            "Open": minute_data['o'],
            "High": minute_data['h'],
            "Low": minute_data['l'],
            "Close": minute_data['c'],
            "Volume": minute_data['v']
        }
        ohlcv_data.append(row)

df = pd.DataFrame(ohlcv_data)
df.to_parquet("backtest_data_step1.parquet", index=False)

# Saving missing tickers
with open("missing_tickers.json", "w") as f:
    json.dump(missing_tickers, f, indent=4)


print(f"Total tickers processed: {len(tickers)}")
print(f"Total tickers missing data: {len(missing_tickers)}")
print(f"Tickers with data from start date: {len(available_start_date)}")
print(f"Tickers with data to end date: {len(available_end_date)}")

### Step 2: Computing previous session High/Low

In [ ]:
# OHLCV data from Step 1
df = pd.read_parquet("backtest_data_step1.parquet")

# Sorting data
df.sort_values(by=["Ticker", "Date", "Time"], inplace=True)

# Computing the daily high and low per ticker
daily_high_low = df.groupby(["Ticker", "Date"]).agg(
    Prev_Session_High=("High", "max"),
    Prev_Session_Low=("Low", "min")
).reset_index()

# Ensuring previous session values align with actual trading days
daily_high_low["Prev_Trading_Date"] = daily_high_low.groupby("Ticker")["Date"].shift(1)

# Forward-filling missing previous session dates (handles non-continuous trading)
daily_high_low["Prev_Trading_Date"] = daily_high_low.groupby("Ticker")["Prev_Trading_Date"].ffill()

# Merging previous session high/low back to the main dataset
df = df.merge(
    daily_high_low[["Ticker", "Date", "Prev_Trading_Date"]],
    on=["Ticker", "Date"],
    how="left"
)

# Merging again to get the correct high/low from the actual previous session
df = df.merge(
    daily_high_low[["Ticker", "Date", "Prev_Session_High", "Prev_Session_Low"]],
    left_on=["Ticker", "Prev_Trading_Date"],
    right_on=["Ticker", "Date"],
    how="left",
    suffixes=("", "_Prev")
)

# Dropping extra columns
df.drop(columns=["Prev_Trading_Date", "Date_Prev"], inplace=True)

# Dropping the first available date since it has NaN values for previous session data
first_day = df["Date"].min()
df = df[df["Date"] != first_day]

# Saving
df.to_parquet("backtest_data_step2.parquet", index=False)

### Step 3: Estimating Bid-Ask spread and OBD

In [ ]:
def compute_bid_ask_spread(df):
    df["Bid-Ask Spread (Estimated)"] = df["High"] - df["Low"]
    df["Bid-Ask Spread (Estimated)"] = df["Bid-Ask Spread (Estimated)"].apply(lambda x: max(x, 0.0001))  # Use a more realistic min value

    return df

def compute_obd(df):
    if "Bid-Ask Spread (Estimated)" not in df.columns:
        df = compute_bid_ask_spread(df)
    df["Estimated OBD"] = df["Volume"] / df["Bid-Ask Spread (Estimated)"]
    return df

df = pd.read_parquet("backtest_data_step2.parquet")
df = compute_bid_ask_spread(df)
df = compute_obd(df)
df.to_parquet("backtest_data_step3.parquet", index=False)

### Step 4: Computing 50-day SMA using daily close prices

In [ ]:
df = pd.read_parquet("backtest_data_step3.parquet")

# Converting Date column to datetime
df["Date"] = pd.to_datetime(df["Date"])

# Creating a DataFrame with only daily closing prices
daily_closes = df.groupby(["Ticker", "Date"])["Close"].last().reset_index()

# Computing 50-day SMA using rolling window on daily data
daily_closes["50-day SMA"] = daily_closes.groupby("Ticker")["Close"].transform(lambda x: x.rolling(window=50, min_periods=1).mean())

# Merging the SMA values back into the original intraday DataFrame
df = df.merge(daily_closes[["Ticker", "Date", "50-day SMA"]], on=["Ticker", "Date"], how="left")

# Dropping extra early dates
df = df[df["Date"] >= "2020-02-17"]


df.to_parquet("backtest_data_step4.parquet", index=False)